In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM
import matplotlib.pyplot as plt

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\Users\David\anaconda3\lib\site-packages\google\protobuf\internal\__init__.py)

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [9]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
#Function to calculate date based on the given number
def calculate_date(row):
    base_date = datetime(2023, 3, 1)  # March 1st, 2023
    delta = timedelta(days=row['week'] - 1)
    return base_date + delta

In [2]:
def generar_predicciones(model,train_MRNN_scN,test_MRNN_scN,Y_scaler,n_input,n_features):
    test_predictions = []
    test_variables=test_MRNN_scN
    temporal=[]

    first_eval_batch = train_MRNN_scN[-n_input:]
    current_batch = first_eval_batch.reshape((1, n_input, n_features))
    test_batch= test_variables.reshape((1,len(test_variables),n_features))

    for i in range(len(test_MRNN_scN)):
        # get the prediction value for the first batch
        current_pred = model.predict(current_batch)[0]
    
        # append the prediction into the array
        test_predictions.append(current_pred[n_input-1,0])

        test_batch[0,i,0]=current_pred[n_input-1,0]
        #test_batch[0,i,0]=current_pred
        temporal= test_batch[0,i,:]
        temporal= temporal.reshape((1,1,6))

        # use the prediction to update the batch and remove the first value
        #current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
        current_batch = np.append(current_batch[:,1:,:],temporal,axis=1)
    
    true_predictions = Y_scaler.inverse_transform([test_predictions])
    Predicciones=[]

    for i in range(len(test_MRNN_scN)):
        Predicciones.append(true_predictions[0][i])

    return Predicciones

In [11]:
df_multivariable[df_multivariable['meal_id'] == 1878]

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
50,1386769,1,24,1878,281.33,280.33,0,0,95
94,1351149,1,11,1878,281.33,282.33,0,0,68
135,1233755,1,83,1878,241.53,242.53,0,0,311
174,1355989,1,32,1878,241.53,241.53,0,0,109
211,1063247,1,13,1878,280.33,280.33,0,0,257
...,...,...,...,...,...,...,...,...,...
456299,1201773,145,132,1878,319.13,319.13,0,0,94
456348,1284991,145,23,1878,321.13,321.13,0,0,41
456433,1384098,145,68,1878,319.13,321.13,0,0,94
456477,1116787,145,51,1878,319.13,321.13,0,0,67


In [12]:
df_multivariable = pd.read_csv('data/train.csv')
meals_list = [1878, 1754, 1558, 2581, 1962, 2104, 1571, 2444]
filtered_df = df_multivariable[df_multivariable['meal_id'].isin(meals_list)]

In [15]:
df_multivariable = pd.read_csv('data/train.csv')
meals_list = [1878, 1754, 1558, 2581, 1962, 2104, 1571, 2444]
df_multivariable = df_multivariable[df_multivariable['meal_id'].isin(meals_list)]

In [5]:
df_multivariable['date'] = df_multivariable.apply(calculate_date, axis=1)
df_multivariable.set_index('date',inplace=True)
df_real_data = df_multivariable.copy(deep=True)

In [13]:
X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()
X_data=X_scaler.fit_transform(df_multivariable.drop(columns=['num_orders']))
Y_data = Y_scaler.fit_transform(df_multivariable[['num_orders']])

In [14]:
df_multivariable[['id', 'week',	'center_id', 'meal_id', 'checkout_price', 'base_price', 'emailer_for_promotion', 'homepage_featured']] = X_data
df_multivariable[['num_orders']] = Y_data

In [15]:
df_multivariable

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
date,,,,,,,,,
2023-03-01,0.759122,0.0,0.255682,0.434530,0.155056,0.119543,0.0,0.0,0.006753
2023-03-01,0.933930,0.0,0.255682,0.491552,0.155056,0.099245,0.0,0.0,0.010582
2023-03-01,0.693979,0.0,0.255682,0.779831,0.152774,0.099282,0.0,0.0,0.007247
2023-03-01,0.676465,0.0,0.255682,0.568638,0.389818,0.471292,0.0,0.0,0.001688
2023-03-01,0.896982,0.0,0.255682,0.828405,0.278617,0.230787,0.0,0.0,0.001112
...,...,...,...,...,...,...,...,...,...
2023-07-23,0.542653,1.0,0.289773,0.253960,0.557303,0.528708,0.0,0.0,0.002265
2023-07-23,0.124072,1.0,0.289773,0.655755,0.554987,0.526242,0.0,0.0,0.001194
2023-07-23,0.221698,1.0,0.289773,0.845829,0.271875,0.327677,0.0,0.0,0.020094


In [16]:
train_MRNN_sc=df_multivariable['2023-03-01':'2023-06-30']
test_MRNN_sc=df_multivariable['2023-07-01':]

In [17]:
train_MRNN_scN=train_MRNN_sc.to_numpy()
test_MRNN_scN=test_MRNN_sc.to_numpy()
n_input = int(7)
n_features = 9
generator = TimeseriesGenerator(train_MRNN_scN, train_MRNN_scN, length=n_input, batch_size=1)   

In [18]:
# define model
model = Sequential()
model.add(LSTM(64, return_sequences=True,activation='relu', input_shape=(n_input, n_features)))
model.add(LSTM(128, return_sequences=True,activation='relu'))
model.add(LSTM(256, return_sequences=True,activation='relu'))
model.add(LSTM(128, return_sequences=True,activation='relu'))
model.add(LSTM(64, return_sequences=True,activation='relu'))
#model.add(Dense(1,activation='linear'))
model.add(LSTM(n_features, return_sequences=True)) 

KeyboardInterrupt: 

In [ ]:
model.compile(optimizer='adam', loss='mse')
history=model.fit(generator,epochs=200)

Epoch 1/200
 10136/381048 [..............................] - ETA: 36:27 - loss: 0.0513

KeyboardInterrupt: 

In [ ]:
mse = history.history['loss']

epochs=range(len(mse)) # Get number of epochs
#------------------------------------------------
# Plot MAE, MSE and Loss
#------------------------------------------------
plt.figure(figsize=(10, 6));
plt.plot(epochs, mse, 'green', label='MSE')
plt.title("Number of epochs vs MSE")
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.legend();

In [ ]:
predicciones=generar_predicciones(model,train_MRNN_scN,test_MRNN_scN,Y_scaler,n_input,n_features)

testinverse=pd.DataFrame(Y_scaler.inverse_transform(test_MRNN_sc), index=test_MRNN_sc.index,columns=test_MRNN_sc.columns)
testinverse['Predictions']=predicciones

mse = mean_squared_error(df_real_data['BMW']['2021-07-02':'2021-12-31'], testinverse['Predictions'])

In [ ]:
df_test = pd.read_csv('data/test.csv')
# Apply the function to create the 'Date' column
df_test['date'] = df_test.apply(calculate_date, axis=1)